In [1]:
!pip install torchvision

In [0]:
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as data
from torch.autograd import Variable

In [0]:
train_dataset=data.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)

In [4]:
(train_dataset[0][0].size())

torch.Size([1, 28, 28])

In [0]:
test_dataset=data.MNIST(root='./data',train=False,transform=transforms.ToTensor())

In [0]:
batch_size=100

In [0]:
n_iters = 3000 

In [0]:
num_epochs=n_iters/(len(train_dataset)/batch_size)

In [0]:
num_epochs=int(num_epochs)

In [0]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

In [0]:
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [0]:
class RNNModel(nn.Module):
  def __init__(self,input_dim,hidden_dim,layer_dim,output_dim):
    super(RNNModel,self).__init__()
    self.hidden_dim=hidden_dim
    self.layer_dim=layer_dim
    self.rnn=nn.RNN(input_dim,hidden_dim,layer_dim,batch_first=True,nonlinearity='relu')
    self.linear=nn.Linear(hidden_dim,output_dim)
  def forward(self,x):
    h0=Variable(torch.zeros(self.layer_dim,x.size(0),self.hidden_dim))
    out,hn=self.rnn(x,h0)
    #getting the last layer
    out =self.linear(out[:,-1,:])
    return out

In [0]:
input_dim=28
hidden_dim=100
layer_dim=1
output_dim=10

In [0]:
model=RNNModel(input_dim,hidden_dim,layer_dim,output_dim)

In [0]:
criterion=nn.CrossEntropyLoss()

In [0]:
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [27]:
len(list(model.parameters()))

6

In [28]:
list(model.parameters())[0].size()

torch.Size([100, 28])

In [29]:
list(model.parameters())[1].size()

torch.Size([100, 100])

In [30]:
list(model.parameters())[2].size()

torch.Size([100])

In [32]:
seq_dim=28

iter=0

for epoch in range(num_epochs):
  for i,(images,labels) in enumerate (train_loader):
    images=Variable(images.view(-1,seq_dim,input_dim))
    labels=Variable(labels)
    optimizer.zero_grad()
    outputs=model(images)
    loss=criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    iter=iter+1
    if(iter%500==0):
      correct=0
      total=0
      for images,labels in test_loader:
        images=Variable(images.view(-1,seq_dim,input_dim))
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        #print(type(predicted))
        total=total+labels.size(0)
        correct=correct+(predicted==labels).sum()
        accuracy=float (100*correct/total)
        #print(correct.numpy())
        #print(total)
        #print(type(accuracy))
        #break
      print("Iteration {}. Loss {}. Accuracy {:{}f} ".format(iter,loss.data[0],accuracy,0.2))
      #break

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Iteration 500. Loss 0.8188353776931763. Accuracy 76.00 
Iteration 1000. Loss 0.40140804648399353. Accuracy 82.00 
Iteration 1500. Loss 0.41714099049568176. Accuracy 83.00 
Iteration 2000. Loss 0.3497374653816223. Accuracy 87.00 
Iteration 2500. Loss 0.3314121663570404. Accuracy 89.00 
Iteration 3000. Loss 0.40200358629226685. Accuracy 91.00 
